In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Activation, Flatten, Reshape, Lambda, Input, concatenate
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
K=tf.keras.backend


In [ ]:

Encoder= Sequential()
Encoder.add( Conv2D(12, (3,3), strides=(2,2), activation='relu', padding='same' , input_shape=(512,512,3) ) ) #256x256x12
Encoder.add( Conv2D(24, (3,3), strides=(2,2), activation='relu', padding='same') ) # 128x128x24
Encoder.add( Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same') ) # 64x64x24
Encoder.add( Conv2D(96, (3,3), strides=(2,2), activation='relu', padding='same') ) # 32x32x48
Encoder.add( Conv2D(128, (3,3), strides=(2,2), activation='relu', padding='same') ) # 16x16x92
Encoder.add( Conv2D(256, (3,3), strides=(2,2), activation='relu', padding='same') ) # 8x8x92
Encoder.add( Conv2D(512, (3,3), strides=(2,2), activation='relu', padding='same') ) # 4x4x512
Encoder.add(Flatten())


Generador = Sequential()
Generador.add(Reshape((4,4,512),input_shape=(8192,)))
Generador.add(Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', activation='relu')) #8x8
Generador.add(Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu')) #16x16
Generador.add(Conv2DTranspose(96, (3,3), strides=(2,2), padding='same', activation='relu')) #32x32
Generador.add(Conv2DTranspose(48, (3,3), strides=(2,2), padding='same', activation='relu')) #64x64
Generador.add(Conv2DTranspose(24, (3,3), strides=(2,2), padding='same', activation='relu')) #128x128
Generador.add(Conv2DTranspose(12, (3,3), strides=(2,2), padding='same', activation='relu')) #256x256
Generador.add(Conv2DTranspose(3, (3,3), strides=(2,2), padding='same', activation='sigmoid')) #512x512x3


Autoencoder=Sequential()
Autoencoder.add(Encoder)
Autoencoder.add(Generador)


In [ ]:
Encoder.summary()
Generador.summary()

In [ ]:
import gc
LR=0.001 #0.001

optimizer0 = RMSprop(learning_rate=LR, epsilon=1e-10)
Autoencoder.compile(loss='mse', optimizer=optimizer0, metrics='mse')

Autoencoder.summary()

In [ ]:
DatosOut=np.zeros( (13500,512,512,3), dtype='float32')

In [ ]:
for z in range(0,50):
    for it in range(0,100):
        DatosOut[:,:,:,0:1]=np.load('Datos_P/Datos_Out_'+ str(it)+'.npz')['arr_0']
        DatosOut[:,:,:,1]=DatosOut[:,:,:,0]//2
        DatosOut[:,:,:,2]=DatosOut[:,:,:,1]//2
        DatosOut[:,:,:,0]=DatosOut[:,:,:,0]-DatosOut[:,:,:,1]*2
        DatosOut[:,:,:,1]=DatosOut[:,:,:,1]-DatosOut[:,:,:,2]*2

        gc.collect()

        for it2 in range(0,25):
            Hist=Autoencoder.fit( DatosOut[np.mod(it2,25)::25,:,:,:], DatosOut[np.mod(it2,25)::25,:,:,:] , batch_size=20, epochs=1, verbose=True)
            print(it2,Hist.history['loss'][0], LR)
            LR=LR*0.999
            K.set_value(Autoencoder.optimizer.lr,LR)
            gc.collect()

In [ ]:
DatosOut_Test=np.zeros( (4500,512,512,3), dtype='float32')
it=0
DatosOut_Test[:,:,:,0:1]=np.load('Datos_P/Datos_Out_Test_'+ str(it)+'.npz')['arr_0']
DatosOut_Test[:,:,:,1]=DatosOut_Test[:,:,:,0]//2
DatosOut_Test[:,:,:,2]=DatosOut_Test[:,:,:,1]//2
DatosOut_Test[:,:,:,0]=DatosOut_Test[:,:,:,0]-DatosOut_Test[:,:,:,1]*2
DatosOut_Test[:,:,:,1]=DatosOut_Test[:,:,:,1]-DatosOut_Test[:,:,:,2]*2

In [ ]:
for q in range(0,10):
    r1=np.random.randint(DatosOut_Test.shape[0])
    pr=Autoencoder.predict( DatosOut_Test[r1:r1+1,:,:,:])
    plt.figure()
    plt.imshow(np.concatenate([pr[0,:,:,0], DatosOut_Test[r1,:,:,0] ], axis=1))
    plt.figure()
    plt.imshow(np.concatenate([pr[0,:,:,1], DatosOut_Test[r1,:,:,1] ], axis=1))
    plt.figure()
    plt.imshow(np.concatenate([pr[0,:,:,2], DatosOut_Test[r1,:,:,2] ], axis=1))
    plt.show()
    plt.pause(0.1)